In [22]:
import csv
import networkx as nx
import numpy as np
import pandas as pd
import array
import tempfile
import simplestatistics as ss
import urllib
%matplotlib inline
# library for displaying plots
import matplotlib.pyplot as plt
# display plots in the notebook
# if this is not set, you will not see the graphic here
import seaborn as sns
from scipy import stats
import matplotlib as mpl
from random import randint
from enum import Enum

class Soil(Enum):
    WOOD = 1
    CERAMICS = 2
    MARBLE = 3
    CONCRETE = 4
    
class Cloth(Enum):
    DRESS = 1
    SKIRT = 2
    SHORT = 3
    JEANWOMAN = 4
    PANT = 5
    JEANMAN = 6
    
#Exits
EXIT_FIRST_FLOOR = 44 # 45 IN THE PICTURE
EXIT_SECOND_FLOOR = 29 # 30 IN THE PICTURE
EXIT_GROUND_FLOOR = 64 # 65 IN THE PICTURE

#Offices
OFFICE_FIRST_FLOOR = 20
OFFICE_SECOND_FLOOR = 9
OFFICE_GROUND_FLOOR = 42

#Earthquake Magnitude
EARTHQUAKE = 6.4711380880 #(2)5.7565922936 #(1)4.0186480949

#Type of speed
NORMAL = 0
FAST = 1

#Save Average Time
times_normal_men = array.array('f',(0.0 for i in range(0,24)))
times_fast_men = array.array('f',(0.0 for i in range(0,24)))
times_normal_women = array.array('f',(0.0 for i in range(0,24)))
times_fast_women = array.array('f',(0.0 for i in range(0,24)))
# Cant People walking on a different superfice
cant_men_soil = array.array('f',(0.0 for i in range(0,24)))
cant_women_soil = array.array('f',(0.0 for i in range(0,24)))
# dataset
dataset = []

# Number of Person (Sex)
MALE = 1
FEMALE = 2
CANT_MALE = 0
CANT_FEMALE = 0

#People for the simulation
COLUMNS = ["HHX","FMX","FPX","SEX","BMI","SLEEP","educ","height","weight"]
people_csv = pd.read_csv("people.csv", sep=",", names=COLUMNS, skipinitialspace=True)
#Convert file people to csv
#people_csv = pd.read_csv(people_data, names=COLUMNS, skipinitialspace=True, skiprows=1)
people = people_csv.as_matrix()
#Put a person in the different offices 
peopleInOfficeOne = array.array('i',(0 for i in range(0,OFFICE_FIRST_FLOOR)))
peopleInOfficeTwo = array.array('i',(0 for i in range(0,OFFICE_SECOND_FLOOR)))
peopleInOfficeGnd = array.array('i',(0 for i in range(0,OFFICE_GROUND_FLOOR)))


#earthquakes = np.fromfile('sim_quakes.dat',dtype=float)
#print earthquakes

class MyRand(object):
    def __init__(self):
        self.last = None

    def __call__(self):
        r = randint(0, 2)
        while r == self.last:
            r = randint(0, 2)
        self.last = r
        return r
    
# Get the floor of the building
# 0: Ground Floor
# 1: First Floor
# 2: Second Floor
def getFloor():
    return randint(0,2);

# Get Office of the floor 
def getOfficeFloor(floor):
    if (floor == 0):# First Floor
        return randint(0,OFFICE_FIRST_FLOOR -1)
    elif(floor == 1):# Second Floor
        return randint(0,OFFICE_SECOND_FLOOR -1)
    elif(floor == 2):# Ground Floor
        return randint(0,OFFICE_GROUND_FLOOR -1)
    return 0

# Set people in the building's office
def setPeopleInOffice(floor, office):
    if (floor == 0): # First Floor
        peopleInOfficeOne[office] = peopleInOfficeOne[office] + 1
    elif (floor == 1):# Second Floor
        peopleInOfficeTwo[office] = peopleInOfficeTwo[office] + 1
    elif (floor == 2):# Ground Floor
        peopleInOfficeGnd[office] = peopleInOfficeGnd[office] + 1

# Get First Floor Graph
def getGraphFFloor():
    reader = csv.reader(open("floor_one.csv", "rb"), delimiter=";")
    x = list(reader)
    ffloor = np.array(x).astype("float")
    return nx.from_numpy_matrix(ffloor, create_using=nx.DiGraph())


# Get Second Floor Graph
def getGraphSFloor():
    reader1 = csv.reader(open("floor_two.csv", "rb"), delimiter=";")
    y = list(reader1)
    sfloor = np.array(y).astype("float")
    return nx.from_numpy_matrix(sfloor, create_using=nx.DiGraph())

# Get Ground Floor Graph
def getGraphGFloor():
    reader2 = csv.reader(open("floor_ground.csv", "rb"), delimiter=";")
    z = list(reader2)
    gfloor = np.array(z).astype("float")
    return nx.from_numpy_matrix(gfloor, create_using=nx.DiGraph())


# Get the way of a person of exit to the building
def getPathofPerson(floor, office):
    if (floor == 0): # First Floor
        return nx.bidirectional_dijkstra(graph_floor_one, office, EXIT_FIRST_FLOOR)
    elif (floor == 1):# Second Floor
        return nx.bidirectional_dijkstra(graph_floor_two, office, EXIT_SECOND_FLOOR)
    elif (floor == 2):# Ground Floor
        return nx.bidirectional_dijkstra(graph_floor_gnd, office, EXIT_GROUND_FLOOR)
    return 0

def getTypeofSoil(floor,node):
    if (floor == 0): #First Floor
        if (node >=0 and node <33):
            return Soil.CERAMICS
        elif (node >= 33 and node <36):
            return Soil.WOOD
        elif (node == 36 or node == 37):
            return Soil.MARBLE
        elif (node >37 and node <44):
            return Soil.CONCRETE
    elif (floor == 1): # Second Floor
        if (node >=0 and node <16):
            return Soil.CERAMICS
        elif (node >= 16 and node <=20):
            return Soil.WOOD
        elif (node == 21 or node == 22):
            return Soil.MARBLE
        elif (node >22 and node <29):
            return Soil.CONCRETE
    elif (floor == 2): # Ground Floor
        if (node >=0 and node <56):
            return Soil.CERAMICS
        elif (node == 56 or node == 57):
            return Soil.MARBLE
        elif (node >=58 and node <64):
            return Soil.CONCRETE
    return Soil.CERAMICS

def getVelocityXSoilBMIMan(speed,soil,bmi):
    signo = randint(0,1);
    incremento = 0.02
    if signo == 1:
        incremento = -0.02
    if (speed == NORMAL): # Normal Speed
        if (bmi < 20.0):
            if (soil == Soil.WOOD):
                return 1.45 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.47 + incremento
            elif (soil == Soil.MARBLE):
                return 1.48 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.50 + incremento
        elif (bmi >= 20 and bmi < 25):
            if (soil == Soil.WOOD):
                return 1.42 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.49 + incremento
            elif (soil == Soil.MARBLE):
                return 1.46 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.49 + incremento 
        elif (bmi >= 25 and bmi < 30):
            if (soil == Soil.WOOD):
                return 1.40 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.44 + incremento
            elif (soil == Soil.MARBLE):
                return 1.43 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.45 + incremento 
        elif (bmi >= 30.0 and bmi < 35):
            if (soil == Soil.WOOD):
                return 1.35 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.39 + incremento
            elif (soil == Soil.MARBLE):
                return 1.38 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.41 + incremento 
        elif (bmi >= 35 and bmi < 40):
            if (soil == Soil.WOOD):
                return 1.27 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.31 + incremento
            elif (soil == Soil.MARBLE):
                return 1.30 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.33 + incremento 
        elif (bmi > 40):
            if (soil == Soil.WOOD):
                return 1.26 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.30 + incremento
            elif (soil == Soil.MARBLE):
                return 1.29 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.31 + incremento 
    elif (speed == FAST): # Fast Speed
        if (bmi < 20.0):
            if (soil == Soil.WOOD):
                return 1.74 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.78 + incremento
            elif (soil == Soil.MARBLE):
                return 1.77 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.79 + incremento
        elif (bmi >= 20 and bmi < 25):
            if (soil == Soil.WOOD):
                return 1.7 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.74 + incremento
            elif (soil == Soil.MARBLE):
                return 1.73 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.75 + incremento 
        elif (bmi >= 25 and bmi < 30):
            if (soil == Soil.WOOD):
                return 1.65 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.69 + incremento
            elif (soil == Soil.MARBLE):
                return 1.68 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.70 + incremento 
        elif (bmi >= 30.0 and bmi < 35):
            if (soil == Soil.WOOD):
                return 1.59 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.63 + incremento
            elif (soil == Soil.MARBLE):
                return 1.62 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.65 + incremento 
        elif (bmi >= 35 and bmi < 40):
            if (soil == Soil.WOOD):
                return 1.55 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.59 + incremento
            elif (soil == Soil.MARBLE):
                return 1.58 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.60 + incremento 
        elif (bmi > 40):
            if (soil == Soil.WOOD):
                return 1.5 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.54 + incremento
            elif (soil == Soil.MARBLE):
                return 1.53 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.55 + incremento
    return 1
        

def getVelocityXSoilBMIWomen(speed,soil,bmi):
    signo = randint(0,1);
    incremento = 0.02
    if signo == 1:
        incremento = -0.02
    if (speed == NORMAL): # Normal Speed
        if (bmi < 20.0):
            if (soil == Soil.WOOD):
                return 1.41 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.45 + incremento
            elif (soil == Soil.MARBLE):
                return 1.42 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.46 + incremento
        elif (bmi >= 20 and bmi < 25):
            if (soil == Soil.WOOD):
                return 1.4 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.43 + incremento
            elif (soil == Soil.MARBLE):
                return 1.41 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.45 + incremento 
        elif (bmi >= 25 and bmi < 30):
            if (soil == Soil.WOOD):
                return 1.36 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.4 + incremento
            elif (soil == Soil.MARBLE):
                return 1.37 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.41 + incremento 
        elif (bmi >= 30.0 and bmi < 35):
            if (soil == Soil.WOOD):
                return 1.32 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.35 + incremento
            elif (soil == Soil.MARBLE):
                return 1.33 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.37 + incremento 
        elif (bmi >= 35 and bmi < 40):
            if (soil == Soil.WOOD):
                return 1.24 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.27 + incremento
            elif (soil == Soil.MARBLE):
                return 1.25 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.29 + incremento 
        elif (bmi > 40):
            if (soil == Soil.WOOD):
                return 1.22 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.26 + incremento
            elif (soil == Soil.MARBLE):
                return 1.23 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.27 + incremento 
    elif (speed == FAST): # Fast Speed
        if (bmi < 20.0):
            if (soil == Soil.WOOD):
                return 1.7 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.74 + incremento
            elif (soil == Soil.MARBLE):
                return 1.71 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.75 + incremento
        elif (bmi >= 20 and bmi < 25):
            if (soil == Soil.WOOD):
                return 1.66 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.7 + incremento
            elif (soil == Soil.MARBLE):
                return 1.67 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.71 + incremento 
        elif (bmi >= 25 and bmi < 30):
            if (soil == Soil.WOOD):
                return 1.61 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.65 + incremento
            elif (soil == Soil.MARBLE):
                return 1.62 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.66 + incremento 
        elif (bmi >= 30.0 and bmi < 35):
            if (soil == Soil.WOOD):
                return 1.56 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.59 + incremento
            elif (soil == Soil.MARBLE):
                return 1.57 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.61 + incremento 
        elif (bmi >= 35 and bmi < 40):
            if (soil == Soil.WOOD):
                return 1.51 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.55 + incremento
            elif (soil == Soil.MARBLE):
                return 1.52 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.56 + incremento 
        elif (bmi > 40):
            if (soil == Soil.WOOD):
                return 1.46 + incremento
            elif (soil == Soil.CERAMICS):
                return 1.5 + incremento
            elif (soil == Soil.MARBLE):
                return 1.47 + incremento
            elif (soil == Soil.CONCRETE):
                return 1.51 + incremento
    return 0

def setTimeXBMISoil(bmi, soil, sex, time, array):
    if (sex == FEMALE): 
        if (bmi < 20.0):
            if (soil == Soil.WOOD):
                array[0] = array[0] + time
            elif (soil == Soil.CERAMICS):
                array[1] = array[1] + time
            elif (soil == Soil.MARBLE):
                array[2] = array[2] + time
            elif (soil == Soil.CONCRETE):
                array[3] = array[3] + time
        elif (bmi >= 20 and bmi < 25):
            if (soil == Soil.WOOD):
                array[4] = array[4] + time
            elif (soil == Soil.CERAMICS):
                array[5] = array[5] + time
            elif (soil == Soil.MARBLE):
                array[6] = array[6] + time
            elif (soil == Soil.CONCRETE):
                array[7] = array[7] + time 
        elif (bmi >= 25 and bmi < 30):
            if (soil == Soil.WOOD):
                array[8] = array[8] + time
            elif (soil == Soil.CERAMICS):
                array[9] = array[9] + time
            elif (soil == Soil.MARBLE):
                array[10] = array[10] + time
            elif (soil == Soil.CONCRETE):
                array[11] = array[11] + time 
        elif (bmi >= 30.0 and bmi < 35):
            if (soil == Soil.WOOD):
                array[12] = array[12] + time
            elif (soil == Soil.CERAMICS):
                array[13] = array[13] + time
            elif (soil == Soil.MARBLE):
                array[14] = array[14] + time
            elif (soil == Soil.CONCRETE):
                array[15] = array[15] + time 
        elif (bmi >= 35 and bmi < 40):
            if (soil == Soil.WOOD):
                array[16] = array[16] + time
            elif (soil == Soil.CERAMICS):
                array[17] = array[17] + time
            elif (soil == Soil.MARBLE):
                array[18] = array[18] + time
            elif (soil == Soil.CONCRETE):
                array[19] = array[19] + time 
        elif (bmi > 40):
            if (soil == Soil.WOOD):
                array[20] = array[20] + time
            elif (soil == Soil.CERAMICS):
                array[21] = array[21] + time
            elif (soil == Soil.MARBLE):
                array[22] = array[22] + time
            elif (soil == Soil.CONCRETE):
                array[23] = array[23] + time 
    elif (sex == MALE): # Fast Speed
        if (bmi < 20.0):
            if (soil == Soil.WOOD):
                array[0] = array[0] + time
            elif (soil == Soil.CERAMICS):
                array[1] = array[1] + time
            elif (soil == Soil.MARBLE):
                array[2] = array[2] + time
            elif (soil == Soil.CONCRETE):
                array[3] = array[3] + time
        elif (bmi >= 20 and bmi < 25):
            if (soil == Soil.WOOD):
                array[4] = array[4] + time
            elif (soil == Soil.CERAMICS):
                array[5] = array[5] + time
            elif (soil == Soil.MARBLE):
                array[6] = array[6] + time
            elif (soil == Soil.CONCRETE):
                array[7] = array[7] + time 
        elif (bmi >= 25 and bmi < 30):
            if (soil == Soil.WOOD):
                array[8] = array[8] + time
            elif (soil == Soil.CERAMICS):
                array[9] = array[9] + time
            elif (soil == Soil.MARBLE):
                array[10] = array[10] + time
            elif (soil == Soil.CONCRETE):
                array[11] = array[11] + time
        elif (bmi >= 30.0 and bmi < 35):
            if (soil == Soil.WOOD):
                array[12] = array[12] + time
            elif (soil == Soil.CERAMICS):
                array[13] = array[13] + time
            elif (soil == Soil.MARBLE):
                array[14] = array[14] + time
            elif (soil == Soil.CONCRETE):
                array[15] = array[15] + time 
        elif (bmi >= 35 and bmi < 40):
            if (soil == Soil.WOOD):
                array[16] = array[16] + time
            elif (soil == Soil.CERAMICS):
                array[17] = array[17] + time
            elif (soil == Soil.MARBLE):
                array[18] = array[18] + time
            elif (soil == Soil.CONCRETE):
                array[19] = array[19] + time 
        elif (bmi > 40):
            if (soil == Soil.WOOD):
                array[20] = array[20] + time
            elif (soil == Soil.CERAMICS):
                array[21] = array[21] + time
            elif (soil == Soil.MARBLE):
                array[22] = array[22] + time
            elif (soil == Soil.CONCRETE):
                array[23] = array[23] + time


def setPeopleBySoilAndBMI(bmi, soil, array): 
    if (bmi < 20.0):
        if (soil == Soil.WOOD):
            array[0] = array[0] + 1
        elif (soil == Soil.CERAMICS):
            array[1] = array[1] + 1
        elif (soil == Soil.MARBLE):
            array[2] = array[2] + 1
        elif (soil == Soil.CONCRETE):
            array[3] = array[3] + 1
    elif (bmi >= 20 and bmi < 25):
        if (soil == Soil.WOOD):
            array[4] = array[4] + 1
        elif (soil == Soil.CERAMICS):
            array[5] = array[5] + 1
        elif (soil == Soil.MARBLE):
            array[6] = array[6] + 1
        elif (soil == Soil.CONCRETE):
            array[7] = array[7] + 1 
    elif (bmi >= 25 and bmi < 30):
        if (soil == Soil.WOOD):
            array[8] = array[8] + 1
        elif (soil == Soil.CERAMICS):
            array[9] = array[9] + 1
        elif (soil == Soil.MARBLE):
            array[10] = array[10] + 1
        elif (soil == Soil.CONCRETE):
            array[11] = array[11] + 1 
    elif (bmi >= 30.0 and bmi < 35):
        if (soil == Soil.WOOD):
            array[12] = array[12] + 1
        elif (soil == Soil.CERAMICS):
            array[13] = array[13] + 1
        elif (soil == Soil.MARBLE):
            array[14] = array[14] + 1
        elif (soil == Soil.CONCRETE):
            array[15] = array[15] + 1 
    elif (bmi >= 35 and bmi < 40):
        if (soil == Soil.WOOD):
            array[16] = array[16] + 1
        elif (soil == Soil.CERAMICS):
            array[17] = array[17] + 1
        elif (soil == Soil.MARBLE):
            array[18] = array[18] + 1
        elif (soil == Soil.CONCRETE):
            array[19] = array[19] + 1 
    elif (bmi > 40):
        if (soil == Soil.WOOD):
            array[20] = array[20] + 1
        elif (soil == Soil.CERAMICS):
            array[21] = array[21] + 1
        elif (soil == Soil.MARBLE):
            array[22] = array[22] + 1
        elif (soil == Soil.CONCRETE):
            array[23] = array[23] + 1 
    
    
def getTimePersonPath(floor, path, bmi, speed, sex):
    time = 0.0
    velocity = 0.0
    reduction = EARTHQUAKE * 10 #MOBILIDAD DE LA PERSONAS
    ADRENALINE = "UP"
    if (floor == 0): # First Floor
        size = len(path)
        i = 0
        while(size > i+1):
            soil = getTypeofSoil(floor,path[i])
            distance = graph_floor_one[path[i]][path[i+1]]['weight']
            if (sex == FEMALE):# Female
                velocity = getVelocityXSoilBMIWomen(speed, soil, bmi)
            elif (sex == MALE):# Male
                velocity = getVelocityXSoilBMIMan(speed, soil, bmi)
            ad = randint(0,2)
            if (i == 0):
                if (ad == 0):#Subida de Adrenaline
                    ADRENALINE ="UP"
                    adrenalin = ((velocity*10)/100)
                elif (ad == 1):#B Bajada de Adrenaline
                    ADRENALINE ="DOWN"
                    adrenalin = -((velocity*10)/100)
                elif (ad == 2):
                    ADRENALINE ="NONE"
                    adrenalin = 0.0
            else:
                if (ad == 0):#Subida de Adrenaline
                    adrenalin = ((velocity*10)/100)
                elif (ad == 1):#B Bajada de Adrenaline
                    adrenalin = -((velocity*10)/100)
                elif (ad == 2):
                    adrenalin = 0.0
            mobility = (velocity * reduction)/100
            velocity = velocity - mobility + adrenalin
            timen = (distance/velocity)
            if (sex == MALE):
                if (speed == NORMAL):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_normal_men)
                elif (speed == FAST):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_fast_men)
                setPeopleBySoilAndBMI(bmi, soil, cant_men_soil)
            elif (sex == FEMALE):
                if (speed == NORMAL):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_normal_women)
                elif (speed == FAST):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_fast_women)
                setPeopleBySoilAndBMI(bmi, soil, cant_women_soil)
            time = time + timen
            i = i + 1
    elif (floor ==1): # Second Floor
        size = len(path)
        i = 0
        while(size > i+1):
            soil = getTypeofSoil(floor,path[i])
            distance = graph_floor_two[path[i]][path[i+1]]['weight']
            if (sex == FEMALE):# Female
                velocity = getVelocityXSoilBMIWomen(speed, soil, bmi)
            elif (sex == MALE):# Male
                velocity = getVelocityXSoilBMIMan(speed, soil, bmi)
            mobility = (velocity * reduction)/100
            ad = randint(0,2)
            if (i == 0):
                if (ad == 0):#Subida de Adrenaline
                    ADRENALINE ="UP"
                    adrenalin = ((velocity*10)/100)
                elif (ad == 1):#B Bajada de Adrenaline
                    ADRENALINE ="DOWN"
                    adrenalin = -((velocity*10)/100)
                elif (ad == 2):
                    ADRENALINE ="NONE"
                    adrenalin = 0.0
            else:
                if (ad == 0):#Subida de Adrenaline
                    adrenalin = ((velocity*10)/100)
                elif (ad == 1):#B Bajada de Adrenaline
                    adrenalin = -((velocity*10)/100)
                elif (ad == 2):
                    adrenalin = 0.0
            velocity = velocity - mobility + adrenalin
            timen = (distance/velocity)
            if (sex == MALE):
                if (speed == NORMAL):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_normal_men)
                elif (speed == FAST):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_fast_men)
                setPeopleBySoilAndBMI(bmi, soil, cant_men_soil)
            elif (sex == FEMALE):
                if (speed == NORMAL):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_normal_women)
                elif (speed == FAST):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_fast_women)
                setPeopleBySoilAndBMI(bmi, soil, cant_women_soil)
            time = time + timen
            i = i + 1
    elif (floor == 2): # Ground Floor
        size = len(path)
        i = 0
        while(size > i+1):
            soil = getTypeofSoil(floor,path[i])
            distance = graph_floor_gnd[path[i]][path[i+1]]['weight']
            if (sex == FEMALE):# Female
                velocity = getVelocityXSoilBMIWomen(speed, soil, bmi)
            elif (sex == MALE):# Male
                velocity = getVelocityXSoilBMIMan(speed, soil, bmi)
            mobility = (velocity * reduction)/100
            ad = randint(0,2)
            if (i == 0):
                if (ad == 0):#Subida de Adrenaline
                    ADRENALINE ="UP"
                    adrenalin = ((velocity*10)/100)
                elif (ad == 1):#B Bajada de Adrenaline
                    ADRENALINE ="DOWN"
                    adrenalin = -((velocity*10)/100)
                elif (ad == 2):
                    ADRENALINE ="NONE"
                    adrenalin = 0.0
            else:
                if (ad == 0):#Subida de Adrenaline
                    adrenalin = ((velocity*10)/100)
                elif (ad == 1):#B Bajada de Adrenaline
                    adrenalin = -((velocity*10)/100)
                elif (ad == 2):
                    adrenalin = 0.0
            velocity = velocity - mobility + adrenalin
            timen = (distance/velocity)
            if (sex == MALE):
                if (speed == NORMAL):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_normal_men)
                elif (speed == FAST):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_fast_men)
                setPeopleBySoilAndBMI(bmi, soil, cant_men_soil)
            elif (sex == FEMALE):
                if (speed == NORMAL):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_normal_women)
                elif (speed == FAST):
                    setTimeXBMISoil(bmi, soil, sex, timen, times_fast_women)
                setPeopleBySoilAndBMI(bmi, soil, cant_women_soil)
            time = time + timen
            i = i + 1
           
    return time, ADRENALINE
            
def getNewBMI(weight, height, wadd):
    return ((weight+wadd)/(height*height))*703

def getPoundsClothingWomen(clothing, typeClothing, cloth):
    if(clothing == 0): #FORMAL
        if(typeClothing == 0):
            if(cloth == Cloth.DRESS):
                return 1.39773
            elif (cloth == Cloth.SKIRT):
                return 1.48592
            elif (cloth == Cloth.SHORT):
                return 1.61819
            elif (cloth == Cloth.JEANWOMAN):
                return 2.389811
        elif (typeClothing == 1):
            if(cloth == Cloth.DRESS):
                return 1.75488
            elif (cloth == Cloth.SKIRT):
                return 1.84306
            elif (cloth == Cloth.SHORT):
                return 1.97534
            elif (cloth == Cloth.JEANWOMAN):
                return 2.74696
    elif(clothing == 1): # INFORMAL
        if(typeClothing == 0):
            if (cloth == Cloth.SKIRT):
                return 1.86291
            elif (cloth == Cloth.SHORT):
                return 1.99518
            elif (cloth == Cloth.JEANWOMAN):
                return 2.766801
        elif (typeClothing == 1):
            if (cloth == Cloth.SKIRT):
                return 1.92904
            elif (cloth == Cloth.SHORT):
                return 2.06132
            elif (cloth == Cloth.JEANWOMAN):
                return 2.83294
            
def getPoundsClothingMen(clothing, typeClothing, cloth):
    if(clothing == 0): #FORMAL
        if(typeClothing == 0):
            if(cloth == Cloth.PANT):
                return 4.607661
            elif (cloth == Cloth.JEANMAN):
                return 5.269048
        elif (typeClothing == 1):
            if(cloth == Cloth.PANT):
                return 6.966607
            elif (cloth == Cloth.JEANMAN):
                return 7.627994
    elif(clothing == 1): # INFORMAL
        if(typeClothing == 0):
            if(cloth == Cloth.PANT):
                return 3.946274
            elif (cloth == Cloth.JEANMAN):
                return 4.607661
        elif (typeClothing == 1):
            if(cloth == Cloth.PANT):
                return 3.68172
            elif (cloth == Cloth.JEANMAN):
                return 4.343107
            
# Floor Graph
graph_floor_one = getGraphFFloor()
graph_floor_two = getGraphSFloor()
graph_floor_gnd = getGraphGFloor()


for pio in range(1,people_csv.shape[0]):
    floor = getFloor()
    office = getOfficeFloor(floor)
    setPeopleInOffice(floor,office)
    bmi = float(people[pio][4])
    sex = "Male"
    if (floor == 0): # First Floor
        length_ideal, path = nx.bidirectional_dijkstra(graph_floor_one, office, EXIT_FIRST_FLOOR)
    elif (floor == 1):# Second Floor
        length_ideal, path = nx.bidirectional_dijkstra(graph_floor_two, office, EXIT_SECOND_FLOOR)
    elif (floor == 2):# Ground Floor
        length_ideal, path = nx.bidirectional_dijkstra(graph_floor_gnd, office, EXIT_GROUND_FLOOR)
    
    if (people[pio][3] == "1"):
        CANT_MALE = CANT_MALE + 1
        sex = "Male"
        #bmiflight1 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingMen(0,0,Cloth.PANT))
        #bmiflight2 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingMen(0,0,Cloth.JEANMAN))
        #bmifheavy1 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingMen(0,1,Cloth.PANT))
        #bmifheavy2 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingMen(0,1,Cloth.JEANMAN))
        #bmiilight1 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingMen(1,0,Cloth.PANT))
        #bmiilight2 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingMen(1,0,Cloth.JEANMAN))
        #bmiiheavy1 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingMen(1,1,Cloth.PANT))
        #bmiiheavy2 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingMen(1,1,Cloth.JEANMAN))
        #print "Male Office:",office," floor: ",floor," path:",path
        #bmi = bmiiheavy2
    else:
        CANT_FEMALE = CANT_FEMALE + 1
        sex = "Female"
        #bmiflight1 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(0,0,Cloth.DRESS))
        #bmiflight2 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(0,0,Cloth.SKIRT))
        #bmiflight3 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(0,0,Cloth.SHORT))
        #bmiflight4 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(0,0,Cloth.JEANWOMAN))
        #bmifheavy1 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(0,1,Cloth.DRESS))
        #bmifheavy2 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(0,1,Cloth.SKIRT))
        #bmifheavy3 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(0,1,Cloth.SHORT))
        #bmifheavy4 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(0,1,Cloth.JEANWOMAN))
        #NO#bmiilight1 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(1,0,Cloth.SKIRT)) -- 
        #bmiilight2 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(1,0,Cloth.SHORT))
        #bmiilight3 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(1,0,Cloth.JEANWOMAN))
        #NO#bmiiheavy1 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(1,1,Cloth.SKIRT)) --
        #bmiiheavy2 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(1,1,Cloth.SHORT))
        #bmiiheavy3 = getNewBMI(int(people[pio][8]),int(people[pio][7]),getPoundsClothingWomen(1,1,Cloth.JEANWOMAN))
        #print "Female Office:",office," floor: ",floor," path:",path
        #bmi = bmiiheavy3
    
    times_normal, ADRENALINE = getTimePersonPath(floor, path, bmi, NORMAL, int(people[pio][3]))
    times_fast, ADRENALINE = getTimePersonPath(floor, path, bmi, FAST, int(people[pio][3])) 
    dataset.append([sex,bmi,times_normal,times_fast,ADRENALINE,office,floor,' '.join(str(x) for x in path)])

data = pd.DataFrame(data=dataset, columns=['Sex','BMI','TimeNormal','TimeFast','Adrenaline','Office','Floor','Path'])
print peopleInOfficeOne
print peopleInOfficeTwo
print peopleInOfficeGnd

#Ideal
times_ideal_men_normal = np.divide(times_normal_men,cant_men_soil)
print times_ideal_men_normal, "Suma: ",np.sum(times_ideal_men_normal)
times_ideal_men_fast = np.divide(times_fast_men,cant_men_soil)
print times_ideal_men_fast, "Suma: ", np.sum(times_ideal_men_fast)
times_ideal_women_normal = np.divide(times_normal_women, cant_women_soil)
print times_ideal_women_normal, "Suma: ", np.sum(times_ideal_women_normal)
times_ideal_women_fast = np.divide(times_fast_women, cant_women_soil)
print times_ideal_women_fast, "Suma: ", np.sum(times_ideal_women_fast)

#datasetFinales
data20 = data[(data.BMI < 20)]
data2025 = data[((data.BMI >= 20) & (data.BMI < 25))]
data2530 = data[((data.BMI >= 25) & (data.BMI < 30))]
data3035 = data[((data.BMI >= 30) & (data.BMI < 35))]
data3540 = data[((data.BMI >= 35) & (data.BMI < 40))]
data40 = data[(data.BMI >= 40)]

# Plot the distribution of the dataset
#g = sns.FacetGrid(data20, hue="Sex", size=5)
#g.map(plt.scatter, "Time Speed Normal (s)", "Time Speed Fast (s)", s=100, linewidth=.5, edgecolor="white")
#g.add_legend();
#print data20[data20.Sex == "Male"].min()
#print data20[data20.Sex == "Male"].max()
#print data20[data20.Sex == "Female"].min()
#print data20[data20.Sex == "Female"].max()


#g1 = sns.FacetGrid(data2025, hue="Sex", size=5)
#g1.map(plt.scatter, "Time Speed Normal (s)", "Time Speed Fast (s)", s=100, linewidth=.5, edgecolor="white")
#g1.add_legend();
#print data2025[data2025.Sex == "Male"].min()
#print data2025[data2025.Sex == "Male"].max()
#print data2025[data2025.Sex == "Female"].min()
#print data2025[data2025.Sex == "Female"].max()

#g2 = sns.FacetGrid(data2530, hue="Sex", size=5)
#g2.map(plt.scatter, "Time Speed Normal (s)", "Time Speed Fast (s)", s=100, linewidth=.5, edgecolor="white")
#g2.add_legend();
#print data2530[data2530.Sex == "Male"].min()
#print data2530[data2530.Sex == "Male"].max()
#print data2530[data2530.Sex == "Female"].min()
#print data2530[data2530.Sex == "Female"].max()

#g3 = sns.FacetGrid(data3035, hue="Sex", size=5)
#g3.map(plt.scatter, "Time Speed Normal (s)", "Time Speed Fast (s)", s=100, linewidth=.5, edgecolor="white")
#g3.add_legend();
#print data3035[data3035.Sex == "Male"].min()
#print data3035[data3035.Sex == "Male"].max()
#print data3035[data3035.Sex == "Female"].min()
#print data3035[data3035.Sex == "Female"].max()

#g4 = sns.FacetGrid(data3540, hue="Sex", size=5)
#g4.map(plt.scatter, "Time Speed Normal (s)", "Time Speed Fast (s)", s=100, linewidth=.5, edgecolor="white")
#g4.add_legend();
#print data3540[data3540.Sex == "Male"].min()
#print data3540[data3540.Sex == "Male"].max()
#print data3540[data3540.Sex == "Female"].min()
#print data3540[data3540.Sex == "Female"].max()

#g5 = sns.FacetGrid(data40, hue="Sex", size=5)
#g5.map(plt.scatter, "Time Speed Normal (s)", "Time Speed Fast (s)", s=100, linewidth=.5, edgecolor="white")
#g5.add_legend();
#print data40[data40.Sex == "Male"].max()
#print data40[data40.Sex == "Male"].max()
#print data40[data40.Sex == "Female"].min()
#print data40[data40.Sex == "Female"].max()


#g6 = sns.FacetGrid(data, col="Adrenaline", row="Sex",margin_titles=True)
#g6 = g6.map(plt.scatter, "Time Speed Normal (s)", "Time Speed Fast (s)", edgecolor="w")
#pal = dict(Male="#6495ED", Female="#F08080")
#g6 = sns.lmplot(x="Time Speed Normal (s)", y="Time Speed Fast (s)", col="Sex", hue="Sex", data=data,palette=pal, y_jitter=.02, logistic=True)
#g6.set(xlim=(0, 80), ylim=(-.05, 1.05))
#sns.set(style="darkgrid", color_codes=True)

df1 =  data20[(data20.TimeNormal < 100.0) & (data20.Sex == "Male")]
df2 =  data20[(data20.TimeNormal < 100.0) & (data20.Sex == "Female")]
df3 =  data20[(data20.TimeNormal > 100.0) & (data20.Sex == "Male") ]
df4 =  data20[(data20.TimeNormal > 100.0) & (data20.Sex == "Female")]
df5 =  data20[(data20.TimeFast < 100.0) & (data20.Sex == "Male") ]
df6 =  data20[(data20.TimeFast < 100.0) & (data20.Sex == "Female")]
df7 =  data20[(data20.TimeFast > 100.0) & (data20.Sex == "Male")]
df8 =  data20[(data20.TimeFast > 100.0) & (data20.Sex == "Female")]
print np.std(df1['TimeNormal'].as_matrix())
print np.std(df2['TimeNormal'].as_matrix())
print np.std(df3['TimeNormal'].as_matrix())
print np.std(df4['TimeNormal'].as_matrix())
print np.std(df5['TimeFast'].as_matrix())
print np.std(df6['TimeFast'].as_matrix())
print np.std(df7['TimeFast'].as_matrix())
print np.std(df8['TimeFast'].as_matrix())
df1 =  data2025[(data2025.TimeNormal < 100.0) & (data2025.Sex == "Male")]
df2 =  data2025[(data2025.TimeNormal < 100.0) & (data2025.Sex == "Female")]
df3 =  data2025[(data2025.TimeNormal > 100.0) & (data2025.Sex == "Male") ]
df4 =  data2025[(data2025.TimeNormal > 100.0) & (data2025.Sex == "Female")]
df5 =  data2025[(data2025.TimeFast < 100.0) & (data2025.Sex == "Male") ]
df6 =  data2025[(data2025.TimeFast < 100.0) & (data2025.Sex == "Female")]
df7 =  data2025[(data2025.TimeFast > 100.0) & (data2025.Sex == "Male")]
df8 =  data2025[(data2025.TimeFast > 100.0) & (data2025.Sex == "Female")]
print np.std(df1['TimeNormal'].as_matrix())
print np.std(df2['TimeNormal'].as_matrix())
print np.std(df3['TimeNormal'].as_matrix())
print np.std(df4['TimeNormal'].as_matrix())
print np.std(df5['TimeFast'].as_matrix())
print np.std(df6['TimeFast'].as_matrix())
print np.std(df7['TimeFast'].as_matrix())
print np.std(df8['TimeFast'].as_matrix())
df1 =  data2530[(data2530.TimeNormal < 100.0) & (data2530.Sex == "Male")]
df2 =  data2530[(data2530.TimeNormal < 100.0) & (data2530.Sex == "Female")]
df3 =  data2530[(data2530.TimeNormal > 100.0) & (data2530.Sex == "Male") ]
df4 =  data2530[(data2530.TimeNormal > 100.0) & (data2530.Sex == "Female")]
df5 =  data2530[(data2530.TimeFast < 100.0) & (data2530.Sex == "Male") ]
df6 =  data2530[(data2530.TimeFast < 100.0) & (data2530.Sex == "Female")]
df7 =  data2530[(data2530.TimeFast > 100.0) & (data2530.Sex == "Male")]
df8 =  data2530[(data2530.TimeFast > 100.0) & (data2530.Sex == "Female")]
print np.std(df1['TimeNormal'].as_matrix())
print np.std(df2['TimeNormal'].as_matrix())
print np.std(df3['TimeNormal'].as_matrix())
print np.std(df4['TimeNormal'].as_matrix())
print np.std(df5['TimeFast'].as_matrix())
print np.std(df6['TimeFast'].as_matrix())
print np.std(df7['TimeFast'].as_matrix())
print np.std(df8['TimeFast'].as_matrix())
df1 =  data3035[(data3035.TimeNormal < 100.0) & (data3035.Sex == "Male")]
df2 =  data3035[(data3035.TimeNormal < 100.0) & (data3035.Sex == "Female")]
df3 =  data3035[(data3035.TimeNormal > 100.0) & (data3035.Sex == "Male") ]
df4 =  data3035[(data3035.TimeNormal > 100.0) & (data3035.Sex == "Female")]
df5 =  data3035[(data3035.TimeFast < 100.0) & (data3035.Sex == "Male") ]
df6 =  data3035[(data3035.TimeFast < 100.0) & (data3035.Sex == "Female")]
df7 =  data3035[(data3035.TimeFast > 100.0) & (data3035.Sex == "Male")]
df8 =  data3035[(data3035.TimeFast > 100.0) & (data3035.Sex == "Female")]
print np.std(df1['TimeNormal'].as_matrix())
print np.std(df2['TimeNormal'].as_matrix())
print np.std(df3['TimeNormal'].as_matrix())
print np.std(df4['TimeNormal'].as_matrix())
print np.std(df5['TimeFast'].as_matrix())
print np.std(df6['TimeFast'].as_matrix())
print np.std(df7['TimeFast'].as_matrix())
print np.std(df8['TimeFast'].as_matrix())
df1 =  data3540[(data3540.TimeNormal < 100.0) & (data3540.Sex == "Male")]
df2 =  data3540[(data3540.TimeNormal < 100.0) & (data3540.Sex == "Female")]
df3 =  data3540[(data3540.TimeNormal > 100.0) & (data3540.Sex == "Male") ]
df4 =  data3540[(data3540.TimeNormal > 100.0) & (data3540.Sex == "Female")]
df5 =  data3540[(data3540.TimeFast < 100.0) & (data3540.Sex == "Male") ]
df6 =  data3540[(data3540.TimeFast < 100.0) & (data3540.Sex == "Female")]
df7 =  data3540[(data3540.TimeFast > 100.0) & (data3540.Sex == "Male")]
df8 =  data3540[(data3540.TimeFast > 100.0) & (data3540.Sex == "Female")]
print np.std(df1['TimeNormal'].as_matrix())
print np.std(df2['TimeNormal'].as_matrix())
print np.std(df3['TimeNormal'].as_matrix())
print np.std(df4['TimeNormal'].as_matrix())
print np.std(df5['TimeFast'].as_matrix())
print np.std(df6['TimeFast'].as_matrix())
print np.std(df7['TimeFast'].as_matrix())
print np.std(df8['TimeFast'].as_matrix())
df1 =  data40[(data40.TimeNormal < 100.0) & (data40.Sex == "Male")]
df2 =  data40[(data40.TimeNormal < 100.0) & (data40.Sex == "Female")]
df3 =  data40[(data40.TimeNormal > 100.0) & (data40.Sex == "Male") ]
df4 =  data40[(data40.TimeNormal > 100.0) & (data40.Sex == "Female")]
df5 =  data40[(data40.TimeFast < 100.0) & (data40.Sex == "Male") ]
df6 =  data40[(data40.TimeFast < 100.0) & (data40.Sex == "Female")]
df7 =  data40[(data40.TimeFast > 100.0) & (data40.Sex == "Male")]
df8 =  data40[(data40.TimeFast > 100.0) & (data40.Sex == "Female")]
print np.std(df1['TimeNormal'].as_matrix())
print np.std(df2['TimeNormal'].as_matrix())
print np.std(df3['TimeNormal'].as_matrix())
print np.std(df4['TimeNormal'].as_matrix())
print np.std(df5['TimeFast'].as_matrix())
print np.std(df6['TimeFast'].as_matrix())
print np.std(df7['TimeFast'].as_matrix())
print np.std(df8['TimeFast'].as_matrix())


#g8 = sns.jointplot("BMI","TimeNormal", data=df1, kind="reg",color="b", size=7)
#g9 = sns.jointplot("BMI","TimeNormal", data=df2, kind="reg",color="b", size=7)
#g9 = sns.jointplot("BMI","TimeFast", data=df, kind="reg",color="b", size=7)


array('i', [70, 78, 84, 82, 69, 67, 76, 84, 79, 83, 78, 81, 69, 88, 87, 81, 86, 83, 74, 69])
array('i', [175, 183, 162, 192, 180, 166, 188, 159, 173])
array('i', [45, 40, 30, 24, 35, 48, 36, 51, 42, 34, 47, 41, 41, 34, 34, 36, 39, 41, 25, 53, 32, 41, 31, 29, 38, 38, 36, 40, 39, 35, 30, 45, 33, 45, 42, 40, 48, 39, 35, 44, 43, 33])
[ 17.11315918  10.94221819   9.749503    10.25185721  19.0652537   12.307527
  11.09916452   9.8774228   18.89509235  12.67611275  11.20404444
  10.32100455  19.39734179  13.23970285  11.81347053  10.68938079
  21.31518151  14.33203287  12.57492636  11.48728881  20.91901081
  13.38631991  12.28235228  11.49096589] Suma:  326.43033408
[ 14.45741102   9.39138542   8.65681385   8.43225824  15.7894162
  10.40931356   9.37524104   8.46084268  15.90584845  10.75835917
   9.40374873   8.74821714  16.57430402  11.01345421  10.14515336
   8.99337023  17.50850659  11.61966541  10.46392872   9.39953018
  16.57322532  11.18857407  10.17069165   9.39518351] Suma:  272.8344